In [1]:
import os
import sqlite3
from dask.distributed import Client, LocalCluster

# set up cluster and workers
cluster = LocalCluster(n_workers=4,
threads_per_worker=1,
memory_limit='8GB') #my RAM size
client = Client(cluster)

# have a look at your workers client
import modin.pandas as pd
import seaborn as sns
import numpy as np
import re
#this sets up dask which is a multi processor library for data frames, 
#prob best to change the 64GB to 16GB and
#if this works you could try increasing the number of workers
#SEE PROGRESS OF DASK RUNTIME FIND LIBRARY
#what codes do and why in notebook
#but in dissertation you explain in more detail

In [2]:
def getlongestmatch(seq, umin = '2', umax = '', gapmin = '1', gapmax = '2', repeatmin = '2', repeatmax = ''):
  m = re.finditer('(([C]{' +umin+ ',' +umax+ '}[AGT]{' +gapmin+ ',' +gapmax+ '}){' +repeatmin+ ',' +repeatmax+ '})|(([G]{' +umin+ ',' +umax+ '}[ACT]{' +gapmin+ ',' +gapmax+ '}){' +repeatmin+ ',' +repeatmax+ '})',seq, flags = re.IGNORECASE)
  longest = ''
  lcount = 0
  span = ''
  for match in m:
    lcount += 1
    if len(match.group()) > len(longest):
      longest = match.group() #.group() tells you what exactly they found e.g. CCCCCC
      span = match.span() #tells you where exactly a match is found (startindx,endindex)
  m = re.finditer('(([C]{' +umin+ ',' +umax+ '}[AGT]{' +gapmin+ ',' +gapmax+ '}){' +repeatmin+ ',' +repeatmin+ '})|(([G]{' +umin+ ',' +umax+ '}[ACT]{' +gapmin+ ',' +gapmax+ '}){' +repeatmin+ ',' +repeatmin+ '})',seq, flags = re.IGNORECASE)
  count = 0
  for match in m:
    count += 1
  return longest, len(longest), span, lcount, count
#function to find the longest match of...GC?
#variable m stores function that iterates and finds through a genetic sequence string, from left to right I'm guessing C or G?
#m is short for match 
#print different things!!
#play around with regex and then plot different ones, some with Gs somewith Cs

In [3]:
def shortrepeat(seq, umin = '4', umax = '', gapmin = '1', gapmax = '', repeatmin = '2', repeatmax = ''):
    match = re.search('(([C]{' +umin+ ',' +umax+ '}[ACGT]{' +gapmin+ ',' +gapmax+ '}){' +repeatmin+ ',' +repeatmin+ '})|(([G]{' +umin+ ',' +umax+ '}[ACT]{' +gapmin+ ',' +gapmax+ '}){' +repeatmin+ ',' +repeatmin+ '})',seq, flags = re.IGNORECASE)
    if match is None:
        return ("")
    else:
        return match.group(0)
#    if match:
      #  return match.group(0)
    #else:
     #   return ("")

In [4]:
#to incorporate in the dataframe you make lambda function again

def kmers(tseq,kmer):
    tseq = tseq.lower()
    start = 0
    end = kmer
    gcs = {} #dictionary
    while (end <= len(tseq)):
        tar = tseq[start:end]
        if tar not in gcs:
            gcs[tar] = 0
        gcs[tar] += 1
        start += 1
        end += 1
    return (gcs)

def gcminmax(tseq,win):
  tseq = tseq.lower()
  acgt = True
  if ((tseq.count('a') + tseq.count('c') + tseq.count('g') + tseq.count('t')) != len(tseq)):
    acgt = False
  target = len(tseq)
  start = 0
  end = win
  gcs = []
  mingcs = 1
  maxgcs = 0
  while (end <= target):
    tar = tseq[start:end]
    new = (tar.count('g') + tar.count('c')) / len(tar)
    if new < mingcs:
      mingcs = new
    if new > maxgcs:
      maxgcs = new
    start += 1
    end += 1
  return (mingcs,maxgcs,acgt)

In [5]:
# this function copied from gquadfinder
def WriteSeq(line,liste, LISTE, F, Len ):
    i,k,I=0,0,0
    a=b=LISTE[i]
    MSCORE=[]
    if len(LISTE) > 1:
        c = LISTE[i+1]
        while i < len(LISTE)-2:
            if c == b+1:
                k = k+1
                i = i+1
            else:
                I = I+1
                seq = line[a:a+F+k]
                sequence, liste2 = BaseScore(seq)
                MSCORE.append(abs(round(np.mean(liste2), 2)))
                k = 0
                i = i+1
                a = LISTE[i]
            b = LISTE[i]
            c = LISTE[i+1]
        I=I+1
        seq=line[a:a+F+k+1]
        sequence, liste2 = BaseScore(seq)
        MSCORE.append(abs(round(np.mean(liste2), 2)))
    else:
        I = I+1
        seq = line[a:a+F]
        MSCORE.append(abs(liste[a]))
    return MSCORE
# this function copied from gquadfinder
def BaseScore(line):
    item, liste = 0, []
    while item < len(line):
        if item < len(line) and (line[item] == "G" or line[item] == "g"):
            liste.append(1)
            if item + 1 < len(line) and (line[item + 1] == "G" or line[item + 1] == "g"):
                liste[item] = 2
                liste.append(2)
                if item + 2 < len(line) and (line[item + 2] == "G" or line[item + 2] == "g"):
                    liste[item + 1] = 3
                    liste[item] = 3
                    liste.append(3)
                    if item + 3 < len(line) and (line[item + 3] == "G" or line[item + 3] == "g"):
                        liste[item] = 4
                        liste[item + 1] = 4
                        liste[item + 2] = 4
                        liste.append(4)
                        item = item + 1
                    item = item + 1
                item = item + 1
            item = item + 1
            while item < len(line) and (line[item] == "G" or line[item] == "g"):
                liste.append(4)
                item = item + 1
        elif item < len(line) and line[item] != "G" and line[item] != "g" and line[item] != "C" and line[item] != "c":
            liste.append(0)
            item = item + 1
        elif item < len(line) and (line[item] == "C" or line[item] == "c"):
            liste.append(-1)
            if item + 1 < len(line) and (line[item + 1] == "C" or line[item + 1] == "c"):
                liste[item] = -2
                liste.append(-2)
                if item + 2 < len(line) and (line[item + 2] == "C" or line[item + 2] == "c"):
                    liste[item + 1] = -3
                    liste[item] = -3
                    liste.append(-3)
                    if item + 3 < len(line) and (line[item + 3] == "C" or line[item + 3] == "c"):
                        liste[item] = -4
                        liste[item + 1] = -4
                        liste[item + 2] = -4
                        liste.append(-4)
                        item = item + 1
                    item = item + 1
                item = item + 1
            item = item + 1
            while item < len(line) and (line[item] == "C" or line[item] == "c"):
                liste.append(-4)
                item = item + 1
        else:
            # la fin du la ligne ou y a des entrers
            item = item + 1
    return line, liste
# this function copied from gquadfinder
def CalScore(liste, k):
    Score_Liste=[]
    for i in range(len(liste)-(k-1)):
        j, Sum = 0, 0
        while j < k:
            Sum = Sum + liste[i]
            j = j + 1
            i = i + 1
        Mean=Sum/float(k)
        Score_Liste.append(Mean)
    return Score_Liste
# this function copied from gquadfinder
def GetG4(line, liste, Window, k, Len):
    LG4 = []
    for i in range(len(liste)):
        if liste[i] >= float(Window) or liste[i] <= - float(Window):
            seq = line[i:i+k]
            LG4.append(i)
    return LG4

'''we use this function for the dataframe but we have to keep the other functions as they are used in this function'''
def gquadcheck(sequence):
    minscore = 2
    window = 50
    cseq, scores = BaseScore(sequence)
    score = CalScore(scores, window)
    outG4 = GetG4(sequence, score, minscore, window, len(scores))
    if len(outG4) == 0:
        return False, []
    mscore = WriteSeq(sequence, score, outG4, window, len(scores))
    if len(mscore):
        return True, mscore
    else:
        return False, mscore

In [16]:
con = sqlite3.connect('C:/Users/44795/Documents/release_db_v5.sqlite3')

In [17]:
# df = pd.read_sql_query('SELECT extra_gc.chromosome,extra_gc.start_idx,extra_gc.end_idx,extra_gc.sequence,extra_gc.state_name,extra_gc.gc,extra_gc.gc_min,extra_gc.gc_max,repeats.repeat_seq,repeats.hmm_state_id,repeats.has_gquad,repeats.has_repeats,repeats.n_repeats,repeats.align_seq,repeats.unit_seq from extra_gc left join repeats on (extra_gc.start_idx=repeats.start_idx and extra_gc.chromosome=repeats.chromosome) order by extra_gc.chromosome,extra_gc.start_idx ', con)
df = pd.read_sql('SELECT * from combined order by chromosome,start_idx ', con) #connection
#this took a lot quicker this time-under a minute

In [13]:
#normaldata = df[df['ISTATE'] == 'NORMAL']
normaldata

,chromosome,start_idx,end_idx,sequence,state_name,gc,gc_min,gc_max,repchr,repstart,...,align_seq,unit_seq,ISTATE,region_length,longest,length,span,longCount,shortCount,match.group(0)
3,chr1,16901,17900,GAATGGTGCCAGGGGCAGAGGGGGCAATGCCGGGGCCCAGGTCGGC...,NORMAL,60.0,55.0,70.0,None,NaN,...,None,None,NORMAL,999,,0,,0,0,GGGGCA
5,chr1,19401,21500,TCATGAGGAAAGGGAGGGGGAGGATGTGGGATGGTGGAGGGGCTGC...,NORMAL,53.0,35.0,65.0,None,NaN,...,None,None,NORMAL,2099,CCCCTGCCCCTGT,13,"(1645, 1658)",2,2,GGGGGA
7,chr1,22201,23700,CTTTCCTGCTGGACCTGACCCTGTGTCATGTCACCTTGCTACCACG...,NORMAL,52.0,37.0,65.0,None,NaN,...,None,None,NORMAL,1499,,0,,0,0,GGGGA
9,chr1,23801,26900,GAAGCTGCAGCCCTCCAGCTCGCCTCCCTCAATGGCCACTCCGTGC...,NORMAL,51.0,30.0,65.0,None,NaN,...,None,None,NORMAL,3099,,0,,0,0,CCCCAT
11,chr1,27101,29400,AAAAGAAACAGCTTGAACAAAAAGTGTGTAGGGGAACCGCAAGCGG...,NORMAL,46.0,20.0,79.0,None,NaN,...,None,None,NORMAL,2299,,0,,0,0,GGGGAA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74319,chr9,138283900,138285599,GTAGAGGCAACTGGGTGTCAGGAGACATGATGGAGAGGAAAGCATC...,NORMAL,49.0,33.0,69.0,None,NaN,...,None,None,NORMAL,1699,,0,,0,0,CCCCTG
74322,chr9,138286000,138286099,AGTGAGCTGAGATCGCACCAGTGCACTCCAGCCTGGGCAACAGAGT...,NORMAL,42.0,42.0,42.0,None,NaN,...,None,None,NORMAL,99,,0,,0,0,
74324,chr9,138286200,138294099,GTTCAGCCACACTGCTTTGGTGTCACAACTGCACATTGGTTTCACA...,NORMAL,34.0,14.0,56.0,chr9,138286563.0,...,NO_ALIGN,NO_UNIT,NORMAL,7899,,0,,0,0,CCCCAA
74326,chr9,138294500,138313799,GTATGGATAAGGCCCTGAGTCATCTTCCTGAGCTGAATGATAGTTA...,NORMAL,37.0,20.0,63.0,chr9,138297703.0,...,NO_ALIGN,NO_UNIT,NORMAL,19299,CCCCATAGCCCCCATAATCCCCA,23,"(881, 904)",2,2,GGGGAC


In [17]:
normaldata

,chromosome,start_idx,end_idx,sequence,state_name,gc,gc_min,gc_max,repchr,repstart,...,match.group(0),mingcs,maxgcs,acgt,gquadcheck,gquadscore,gquadscoremax,gquadCount,gquadDensity per kb,gquadscoremin
3,chr1,16901,17900,GAATGGTGCCAGGGGCAGAGGGGGCAATGCCGGGGCCCAGGTCGGC...,NORMAL,60.0,55.0,70.0,None,NaN,...,GGGGCA,0.2,1.0,True,False,[],0.0,0,0.0,0.0
5,chr1,19401,21500,TCATGAGGAAAGGGAGGGGGAGGATGTGGGATGGTGGAGGGGCTGC...,NORMAL,53.0,35.0,65.0,None,NaN,...,GGGGGA,0.0,0.9,True,False,[],0.0,0,0.0,0.0
7,chr1,22201,23700,CTTTCCTGCTGGACCTGACCCTGTGTCATGTCACCTTGCTACCACG...,NORMAL,52.0,37.0,65.0,None,NaN,...,GGGGA,0.0,0.9,True,False,[],0.0,0,0.0,0.0
9,chr1,23801,26900,GAAGCTGCAGCCCTCCAGCTCGCCTCCCTCAATGGCCACTCCGTGC...,NORMAL,51.0,30.0,65.0,None,NaN,...,CCCCAT,0.0,1.0,True,False,[],0.0,0,0.0,0.0
11,chr1,27101,29400,AAAAGAAACAGCTTGAACAAAAAGTGTGTAGGGGAACCGCAAGCGG...,NORMAL,46.0,20.0,79.0,None,NaN,...,GGGGAA,0.0,1.0,True,False,[],0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74319,chr9,138283900,138285599,GTAGAGGCAACTGGGTGTCAGGAGACATGATGGAGAGGAAAGCATC...,NORMAL,49.0,33.0,69.0,None,NaN,...,CCCCTG,0.0,0.9,True,False,[],0.0,0,0.0,0.0
74322,chr9,138286000,138286099,AGTGAGCTGAGATCGCACCAGTGCACTCCAGCCTGGGCAACAGAGT...,NORMAL,42.0,42.0,42.0,None,NaN,...,,0.1,0.8,True,False,[],0.0,0,0.0,0.0
74324,chr9,138286200,138294099,GTTCAGCCACACTGCTTTGGTGTCACAACTGCACATTGGTTTCACA...,NORMAL,34.0,14.0,56.0,chr9,138286563.0,...,CCCCAA,0.0,1.0,True,False,[],0.0,0,0.0,0.0
74326,chr9,138294500,138313799,GTATGGATAAGGCCCTGAGTCATCTTCCTGAGCTGAATGATAGTTA...,NORMAL,37.0,20.0,63.0,chr9,138297703.0,...,GGGGAC,0.0,1.0,True,False,[],0.0,0,0.0,0.0


In [18]:
normaldata.to_pickle('normal.pickle')

In [19]:
normalpickle = pd.read_pickle('normal.pickle')

In [22]:
normalpickle
#pd.set_option('display.max_columns', None)

,chromosome,start_idx,end_idx,sequence,state_name,gc,gc_min,gc_max,repchr,repstart,repend,shortCount,match.group(0),mingcs,maxgcs,acgt,gquadcheck,gquadscore,gquadscoremax,gquadCount,gquadDensity per kb,gquadscoremin
3,chr1,16901,17900,GAATGGTGCCAGGGGCAGAGGGGGCAATGCCGGGGCCCAGGTCGGC...,NORMAL,60.0,55.0,70.0,None,NaN,NaN,0,GGGGCA,0.2,1.0,True,False,[],0.0,0,0.0,0.0
5,chr1,19401,21500,TCATGAGGAAAGGGAGGGGGAGGATGTGGGATGGTGGAGGGGCTGC...,NORMAL,53.0,35.0,65.0,None,NaN,NaN,2,GGGGGA,0.0,0.9,True,False,[],0.0,0,0.0,0.0
7,chr1,22201,23700,CTTTCCTGCTGGACCTGACCCTGTGTCATGTCACCTTGCTACCACG...,NORMAL,52.0,37.0,65.0,None,NaN,NaN,0,GGGGA,0.0,0.9,True,False,[],0.0,0,0.0,0.0
9,chr1,23801,26900,GAAGCTGCAGCCCTCCAGCTCGCCTCCCTCAATGGCCACTCCGTGC...,NORMAL,51.0,30.0,65.0,None,NaN,NaN,0,CCCCAT,0.0,1.0,True,False,[],0.0,0,0.0,0.0
11,chr1,27101,29400,AAAAGAAACAGCTTGAACAAAAAGTGTGTAGGGGAACCGCAAGCGG...,NORMAL,46.0,20.0,79.0,None,NaN,NaN,0,GGGGAA,0.0,1.0,True,False,[],0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74319,chr9,138283900,138285599,GTAGAGGCAACTGGGTGTCAGGAGACATGATGGAGAGGAAAGCATC...,NORMAL,49.0,33.0,69.0,None,NaN,NaN,0,CCCCTG,0.0,0.9,True,False,[],0.0,0,0.0,0.0
74322,chr9,138286000,138286099,AGTGAGCTGAGATCGCACCAGTGCACTCCAGCCTGGGCAACAGAGT...,NORMAL,42.0,42.0,42.0,None,NaN,NaN,0,,0.1,0.8,True,False,[],0.0,0,0.0,0.0
74324,chr9,138286200,138294099,GTTCAGCCACACTGCTTTGGTGTCACAACTGCACATTGGTTTCACA...,NORMAL,34.0,14.0,56.0,chr9,138286563.0,138287563.0,0,CCCCAA,0.0,1.0,True,False,[],0.0,0,0.0,0.0
74326,chr9,138294500,138313799,GTATGGATAAGGCCCTGAGTCATCTTCCTGAGCTGAATGATAGTTA...,NORMAL,37.0,20.0,63.0,chr9,138297703.0,138298703.0,2,GGGGAC,0.0,1.0,True,False,[],0.0,0,0.0,0.0


In [42]:
#normaldata.to_pickle('normal.pickle.gzde', compression='gzip')

In [8]:
normaldata['longest'],normaldata['length'],normaldata['span'], normaldata['longCount'], normaldata['shortCount'] = zip(*normaldata.sequence.apply(lambda x: getlongestmatch(x,'4','','1','5','2','')))
#longest runtime here-only have to execute once
#took like 5-10mins

In [9]:
normaldata['match.group(0)'] = normaldata.sequence.apply(lambda x: shortrepeat(x,'4','','1','2','1',''))
#look at pickle instead of .copy() for this cell

In [10]:
normaldata['mingcs'],normaldata['maxgcs'],normaldata['acgt'] = zip(*normaldata.sequence.apply(lambda x: gcminmax(x, 10)))
#normaldata

In [16]:
normaldata['gquadscoremin'] = normaldata.gquadscore.apply(lambda x: np.amin(x, initial=0))

In [12]:
normaldata['gquadcheck'],normaldata['gquadscore'] = zip(*normaldata.sequence.apply(lambda x: gquadcheck(x)))

#counting the gquadscore array,np.amin, np.amean (look into it) then use to plot boxplot, 
#create boxplot on gquad score to compare each istate
#read more on gquadscore paper sent on teams

In [13]:
normaldata['gquadscoremax'] = normaldata.gquadscore.apply(lambda x: np.amax(x, initial=0))

In [14]:
normaldata['gquadCount'] = normaldata.gquadscore.apply(lambda x: len(x))

In [15]:
#to find density of gquad per row
normaldata['gquadDensity per kb'] = normaldata['gquadCount']/normaldata['region_length'] * 1000

In [ ]:
#create row to show how many gquads in each row, by using len(array)
#show difference between maxscore and minscore between ISTATEs
#find density of gquad per row for each istate-divide count of gquad by region length and multiply by 100 or 1000 to find per 100 or 1000base pair (kb thing) 
#and then plot to compare but make column and add to dataframe ^
#look into statistics for any significant difference like T test

In [23]:
normaldata.describe()

,start_idx,end_idx,gc,gc_min,gc_max,repstart,repend,hmm_state_id,has_gquad,has_repeats,n_repeats,n_repeats,region_length,length,longCount,shortCount,mingcs,maxgcs,gquadscoremax,gquadCount,gquadDensity per kb,gquadscoremin
count,3.195600e+04,3.195600e+04,31956.000000,31956.000000,31956.000000,1.112400e+04,1.112400e+04,11124.0,11124.000000,11124.000000,11124.000000,11124.000000,3.195600e+04,31956.000000,31956.000000,31956.000000,31956.000000,31956.000000,31956.000000,31956.000000,31956.000000,31956.0
mean,6.981964e+07,6.989906e+07,44.252785,25.195550,65.136782,7.322194e+07,7.322294e+07,1.0,0.008270,0.076861,0.546386,0.546386,7.942570e+04,12.030386,15.207066,15.422237,0.029926,0.928702,0.423725,0.643103,0.019346,0.0
std,5.939002e+07,5.940509e+07,8.931012,14.686835,13.246151,5.902003e+07,5.902003e+07,0.0,0.090569,0.266382,2.415953,2.415953,1.688208e+05,11.018369,34.773993,35.290605,0.077601,0.103377,0.815586,1.852000,0.336795,0.0
min,1.030100e+04,1.230000e+04,0.000000,0.000000,0.000000,1.900800e+04,2.000800e+04,1.0,0.000000,0.000000,0.000000,0.000000,9.900000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,1.896472e+07,1.912275e+07,38.000000,15.000000,57.000000,2.290372e+07,2.290472e+07,1.0,0.000000,0.000000,0.000000,0.000000,7.990000e+02,0.000000,0.000000,0.000000,0.000000,0.900000,0.000000,0.000000,0.000000,0.0
50%,5.897070e+07,5.912900e+07,43.000000,23.000000,66.000000,6.301162e+07,6.301262e+07,1.0,0.000000,0.000000,0.000000,0.000000,4.499000e+03,15.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0
75%,1.070836e+08,1.073595e+08,50.000000,34.000000,75.000000,1.119004e+08,1.119014e+08,1.0,0.000000,0.000000,0.000000,0.000000,7.682400e+04,21.000000,14.000000,14.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0
max,2.489379e+08,2.489382e+08,95.000000,95.000000,97.000000,2.487444e+08,2.487454e+08,1.0,1.000000,1.000000,36.000000,36.000000,2.406599e+06,247.000000,632.000000,645.000000,0.700000,1.000000,3.670000,32.000000,30.303030,0.0


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
 
# Make the dataframe for evaluation on Errorbars
df = pd.normaldf({
    'insert': [0.0, 0.1, 0.3, 0.5, 1.0],
    'mean': [0.423725, 0.855006, 0.398355],
    'quality': ['good', 'good', 'poor'],
    'std': [0.815586, 1.056095, 0.805476]})
 
print(df)

In [ ]:
import matplotlib.pyplot as plt
sns.kdeplot(data=normaldata[normaldata["gquadDensity per kb"] == "30.303030"], x="", color='orange')
sns.kdeplot(data=coredata[coredata["gquadDensity per kb"] == "100.250627"], x="region_length", color='orange')
sns.kdeplot(data=tcdata[tcdata["gquadDensity per kb"] == "40.404040"], x="region_length", color='brown')
plt.legend(title='ISTATE', loc='upper right', labels=['TUFCORE', 'TUF','DELETION','DUPLICATION','CORE', 'NORMAL'])
plt.xlim(-4000, 5000)
#density plots are like histograms